<a href="https://colab.research.google.com/github/Augusto-Seixas-UFV/seixas-ufv-iac/blob/main/UFV_ELT573_BostonHousing_Relat%C3%B3rio2025_Seixas_v_Antr_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Análise Completa do Boston Housing Dataset
Disciplina: ELT 573 – Introdução ao Aprendizado Estatístico
Implementação de Regressão Linear, Árvores, Bagging e Random Forest
com Validação Cruzada Leave-One-Out
"""

# =============================================================================
# IMPORTAÇÃO DAS BIBLIOTECAS
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, skew, kurtosis
from scipy import stats
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
np.random.seed(42)

# =============================================================================
# 1. CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================

def load_and_prepare_data():
    """Carrega e prepara o Boston Housing Dataset"""

    print("="*60)
    print("1. CARREGAMENTO DO BOSTON HOUSING DATASET")
    print("="*60)

    # Carregamento do dataset
    boston = load_boston()
    X = pd.DataFrame(boston.data, columns=boston.feature_names)
    y = pd.Series(boston.target, name='MEDV')

    print(f"Dimensões do dataset: {X.shape}")
    print(f"Variáveis explicativas: {list(X.columns)}")
    print(f"Variável alvo (MEDV): min={y.min():.1f}, max={y.max():.1f}, média={y.mean():.1f}")
    print(f"Valores ausentes em X: {X.isnull().sum().sum()}")
    print(f"Valores ausentes em y: {y.isnull().sum()}")

    return X, y, boston

# =============================================================================
# 2. ANÁLISE EXPLORATÓRIA DOS DADOS
# =============================================================================

def exploratory_data_analysis(X, y):
    """Realiza análise exploratória completa dos dados"""

    print("\n" + "="*60)
    print("2. ANÁLISE EXPLORATÓRIA DOS DADOS")
    print("="*60)

    # 2.1 Estatísticas Descritivas
    print("\n2.1 ESTATÍSTICAS DESCRITIVAS")
    print("-" * 40)

    desc_stats = X.describe()
    print("Estatísticas das Variáveis Explicativas:")
    print(desc_stats.round(2))

    print(f"\nEstatísticas da Variável Alvo (MEDV):")
    print(f"Média: {y.mean():.2f}")
    print(f"Desvio Padrão: {y.std():.2f}")
    print(f"Mediana: {y.median():.2f}")
    print(f"Assimetria: {skew(y):.3f}")
    print(f"Curtose: {kurtosis(y):.3f}")

    # 2.2 Análise de Correlações
    print("\n2.2 ANÁLISE DE CORRELAÇÕES")
    print("-" * 40)

    # Matriz de correlação completa
    df_complete = pd.concat([X, y], axis=1)
    correlation_matrix = df_complete.corr()

    # Correlações com MEDV
    correlations_with_target = correlation_matrix['MEDV'].drop('MEDV').sort_values(key=abs, ascending=False)
    print("Correlações com MEDV (ordenadas por magnitude):")
    for var, corr in correlations_with_target.head(10).items():
        print(f"{var:8s}: {corr:6.3f}")

    # 2.3 Visualizações
    create_exploratory_plots(X, y, correlation_matrix, correlations_with_target)

    return correlation_matrix, correlations_with_target

def create_exploratory_plots(X, y, correlation_matrix, correlations_with_target):
    """Cria visualizações para análise exploratória"""

    # Figura 1: Distribuição da variável alvo
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Histograma
    axes[0].hist(y, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].set_title('Distribuição de MEDV')
    axes[0].set_xlabel('Valor Médio das Casas ($1000s)')
    axes[0].set_ylabel('Frequência')
    axes[0].axvline(y.mean(), color='red', linestyle='--', label=f'Média: {y.mean():.1f}')
    axes[0].legend()

    # Box plot
    axes[1].boxplot(y)
    axes[1].set_title('Box Plot - MEDV')
    axes[1].set_ylabel('Valor Médio das Casas ($1000s)')

    # Q-Q Plot
    stats.probplot(y, dist="norm", plot=axes[2])
    axes[2].set_title('Q-Q Plot - Normalidade de MEDV')

    plt.tight_layout()
    plt.show()

    # Figura 2: Matriz de correlação
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, mask=mask, fmt='.2f')
    plt.title('Matriz de Correlação - Boston Housing Dataset')
    plt.tight_layout()
    plt.show()

    # Figura 3: Scatter plots das variáveis mais correlacionadas
    top_vars = correlations_with_target.head(6).index.tolist()
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, var in enumerate(top_vars):
        axes[i].scatter(X[var], y, alpha=0.6, color='steelblue')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('MEDV')
        axes[i].set_title(f'MEDV vs {var}\n(r = {correlation_matrix.loc[var, "MEDV"]:.3f})')

        # Adicionar linha de tendência
        z = np.polyfit(X[var], y, 1)
        p = np.poly1d(z)
        axes[i].plot(X[var], p(X[var]), "r--", alpha=0.8)

    plt.tight_layout()
    plt.show()

# =============================================================================
# 3. IMPLEMENTAÇÃO DOS MODELOS
# =============================================================================

def setup_models():
    """Configura os modelos de machine learning"""

    print("\n" + "="*60)
    print("3. CONFIGURAÇÃO DOS MODELOS")
    print("="*60)

    models = {
        'Regressão Linear': LinearRegression(),
        'Árvore de Regressão': DecisionTreeRegressor(
            random_state=42,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=3
        ),
        'Bagging': BaggingRegressor(
            base_estimator=DecisionTreeRegressor(random_state=42),
            n_estimators=100,
            random_state

SyntaxError: incomplete input (ipython-input-1-1432429431.py, line 177)

In [2]:
"""
Análise Completa do Boston Housing Dataset
Disciplina: ELT 573 – Introdução ao Aprendizado Estatístico
Implementação de Regressão Linear, Árvores, Bagging e Random Forest
com Validação Cruzada Leave-One-Out
"""

# =============================================================================
# IMPORTAÇÃO DAS BIBLIOTECAS
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, skew, kurtosis
from scipy import stats
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
np.random.seed(42)

# =============================================================================
# 1. CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================

def load_and_prepare_data():
    """Carrega e prepara o Boston Housing Dataset"""

    print("="*60)
    print("1. CARREGAMENTO DO BOSTON HOUSING DATASET")
    print("="*60)

    # Carregamento do dataset
    boston = load_boston()
    X = pd.DataFrame(boston.data, columns=boston.feature_names)
    y = pd.Series(boston.target, name='MEDV')

    print(f"Dimensões do dataset: {X.shape}")
    print(f"Variáveis explicativas: {list(X.columns)}")
    print(f"Variável alvo (MEDV): min={y.min():.1f}, max={y.max():.1f}, média={y.mean():.1f}")
    print(f"Valores ausentes em X: {X.isnull().sum().sum()}")
    print(f"Valores ausentes em y: {y.isnull().sum()}")

    return X, y, boston

# =============================================================================
# 2. ANÁLISE EXPLORATÓRIA DOS DADOS
# =============================================================================

def exploratory_data_analysis(X, y):
    """Realiza análise exploratória completa dos dados"""

    print("\n" + "="*60)
    print("2. ANÁLISE EXPLORATÓRIA DOS DADOS")
    print("="*60)

    # 2.1 Estatísticas Descritivas
    print("\n2.1 ESTATÍSTICAS DESCRITIVAS")
    print("-" * 40)

    desc_stats = X.describe()
    print("Estatísticas das Variáveis Explicativas:")
    print(desc_stats.round(2))

    print(f"\nEstatísticas da Variável Alvo (MEDV):")
    print(f"Média: {y.mean():.2f}")
    print(f"Desvio Padrão: {y.std():.2f}")
    print(f"Mediana: {y.median():.2f}")
    print(f"Assimetria: {skew(y):.3f}")
    print(f"Curtose: {kurtosis(y):.3f}")

    # 2.2 Análise de Correlações
    print("\n2.2 ANÁLISE DE CORRELAÇÕES")
    print("-" * 40)

    # Matriz de correlação completa
    df_complete = pd.concat([X, y], axis=1)
    correlation_matrix = df_complete.corr()

    # Correlações com MEDV
    correlations_with_target = correlation_matrix['MEDV'].drop('MEDV').sort_values(key=abs, ascending=False)
    print("Correlações com MEDV (ordenadas por magnitude):")
    for var, corr in correlations_with_target.head(10).items():
        print(f"{var:8s}: {corr:6.3f}")

    # 2.3 Visualizações
    create_exploratory_plots(X, y, correlation_matrix, correlations_with_target)

    return correlation_matrix, correlations_with_target

def create_exploratory_plots(X, y, correlation_matrix, correlations_with_target):
    """Cria visualizações para análise exploratória"""

    # Figura 1: Distribuição da variável alvo
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Histograma
    axes[0].hist(y, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].set_title('Distribuição de MEDV')
    axes[0].set_xlabel('Valor Médio das Casas ($1000s)')
    axes[0].set_ylabel('Frequência')
    axes[0].axvline(y.mean(), color='red', linestyle='--', label=f'Média: {y.mean():.1f}')
    axes[0].legend()

    # Box plot
    axes[1].boxplot(y)
    axes[1].set_title('Box Plot - MEDV')
    axes[1].set_ylabel('Valor Médio das Casas ($1000s)')

    # Q-Q Plot
    stats.probplot(y, dist="norm", plot=axes[2])
    axes[2].set_title('Q-Q Plot - Normalidade de MEDV')

    plt.tight_layout()
    plt.show()

    # Figura 2: Matriz de correlação
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, mask=mask, fmt='.2f')
    plt.title('Matriz de Correlação - Boston Housing Dataset')
    plt.tight_layout()
    plt.show()

    # Figura 3: Scatter plots das variáveis mais correlacionadas
    top_vars = correlations_with_target.head(6).index.tolist()
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, var in enumerate(top_vars):
        axes[i].scatter(X[var], y, alpha=0.6, color='steelblue')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('MEDV')
        axes[i].set_title(f'MEDV vs {var}\n(r = {correlation_matrix.loc[var, "MEDV"]:.3f})')

        # Adicionar linha de tendência
        z = np.polyfit(X[var], y, 1)
        p = np.poly1d(z)
        axes[i].plot(X[var], p(X[var]), "r--", alpha=0.8)

    plt.tight_layout()
    plt.show()

# =============================================================================
# 3. IMPLEMENTAÇÃO DOS MODELOS
# =============================================================================

def setup_models():
    """Configura os modelos de machine learning"""

    print("\n" + "="*60)
    print("3. CONFIGURAÇÃO DOS MODELOS")
    print("="*60)

    models = {
        'Regressão Linear': LinearRegression(),
        'Árvore de Regressão': DecisionTreeRegressor(
            random_state=42,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=3
        ),
        'Bagging': BaggingRegressor(
            base_estimator=DecisionTreeRegressor(random_state=42),
            n_estimators=100,
            random_state=42,
            max_samples=0.8
        ),
        'Random Forest': RandomForestRegressor(
            n_estimators=100,
            random_state=42,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt'
        )
    }

    print("Modelos configurados:")
    for name, model in models.items():
        print(f"- {name}: {type(model).__name__}")
        if hasattr(model, 'get_params'):
            key_params = {k: v for k, v in model.get_params().items()
                         if k in ['max_depth', 'n_estimators', 'random_state']}
            print(f"  Parâmetros principais: {key_params}")

    return models

# =============================================================================
# 4. VALIDAÇÃO CRUZADA LEAVE-ONE-OUT
# =============================================================================

def perform_loocv_analysis(X, y, models):
    """Executa validação cruzada Leave-One-Out para todos os modelos"""

    print("\n" + "="*60)
    print("4. VALIDAÇÃO CRUZADA LEAVE-ONE-OUT")
    print("="*60)

    # Inicialização do validador LOOCV
    loo = LeaveOneOut()
    n_samples = X.shape[0]

    print(f"Executando LOOCV com {n_samples} iterações para cada modelo...")
    print("Isso pode levar alguns minutos...\n")

    # Armazenamento dos resultados
    results = {}

    for name, model in models.items():
        print(f"Processando {name}...")

        # Arrays para armazenar predições
        y_pred = []
        y_true = []

        # Contador para progresso
        count = 0

        # Loop LOOCV
        for train_idx, test_idx in loo.split(X):
            # Divisão treino/teste
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Treinamento e predição
            model.fit(X_train, y_train)
            pred = model.predict(X_test)

            # Armazenamento
            y_pred.extend(pred)
            y_true.extend(y_test)

            # Progresso
            count += 1
            if count % 100 == 0:
                print(f"  Processadas {count}/{n_samples} iterações")

        # Conversão para arrays numpy
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)

        # Cálculo das métricas
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        correlation, p_value = pearsonr(y_true, y_pred)
        mae = np.mean(np.abs(y_true - y_pred))

        # Armazenamento dos resultados
        results[name] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'Correlação': correlation,
            'p_value': p_value,
            'y_pred': y_pred,
            'y_true': y_true
        }

        print(f"  {name} concluído - MSE: {mse:.3f}, RMSE: {rmse:.3f}, r: {correlation:.3f}")

    return results

# =============================================================================
# 5. ANÁLISE DOS RESULTADOS
# =============================================================================

def analyze_results(results):
    """Analisa e apresenta os resultados dos modelos"""

    print("\n" + "="*60)
    print("5. ANÁLISE DOS RESULTADOS")
    print("="*60)

    # 5.1 Tabela de resultados
    print("\n5.1 COMPARAÇÃO DE PERFORMANCE")
    print("-" * 40)

    results_df = pd.DataFrame({
        'Modelo': list(results.keys()),
        'MSE': [results[model]['MSE'] for model in results.keys()],
        'RMSE': [results[model]['RMSE'] for model in results.keys()],
        'MAE': [results[model]['MAE'] for model in results.keys()],
        'Correlação': [results[model]['Correlação'] for model in results.keys()]
    })

    # Ordenar por MSE
    results_df = results_df.sort_values('MSE').reset_index(drop=True)
    results_df.index = results_df.index + 1

    print(results_df.round(3).to_string())

    # Identificar melhor modelo
    best_model = results_df.iloc[0]['Modelo']
    print(f"\nMelhor modelo: {best_model}")
    print(f"MSE: {results_df.iloc[0]['MSE']:.3f}")
    print(f"Correlação: {results_df.iloc[0]['Correlação']:.3f}")

    # 5.2 Visualizações dos resultados
    create_results_visualizations(results, results_df)

    # 5.3 Análise estatística
    perform_statistical_analysis(results)

    return results_df, best_model

def create_results_visualizations(results, results_df):
    """Cria visualizações dos resultados"""

    # Figura 1: Gráficos de dispersão Predito vs Observado
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    for i, (name, result) in enumerate(results.items()):
        y_true = result['y_true']
        y_pred = result['y_pred']

        # Scatter plot
        axes[i].scatter(y_true, y_pred, alpha=0.6, color='steelblue')

        # Linha diagonal perfeita
        min_val = min(min(y_true), min(y_pred))
        max_val = max(max(y_true), max(y_pred))
        axes[i].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Predição Perfeita')

        # Configurações
        axes[i].set_xlabel('Valores Observados')
        axes[i].set_ylabel('Valores Preditos')
        axes[i].set_title(f'{name}\nMSE: {result["MSE"]:.3f}, r: {result["Correlação"]:.3f}')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Figura 2: Gráficos de barras comparativos
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # MSE
    bars1 = axes[0].bar(results_df['Modelo'], results_df['MSE'], color='lightcoral', alpha=0.8)
    axes[0].set_title('Erro Quadrático Médio (MSE)')
    axes[0].set_ylabel('MSE')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)

    # Adicionar valores nas barras
    for bar, value in zip(bars1, results_df['MSE']):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    f'{value:.1f}', ha='center', va='bottom')

    # RMSE
    bars2 = axes[1].bar(results_df['Modelo'], results_df['RMSE'], color='lightgreen', alpha=0.8)
    axes[1].set_title('Raiz do Erro Quadrático Médio (RMSE)')
    axes[1].set_ylabel('RMSE')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)

    for bar, value in zip(bars2, results_df['RMSE']):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                    f'{value:.1f}', ha='center', va='bottom')

    # Correlação
    bars3 = axes[2].bar(results_df['Modelo'], results_df['Correlação'], color='lightblue', alpha=0.8)
    axes[2].set_title('Correlação de Pearson')
    axes[2].set_ylabel('Correlação')
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(True, alpha=0.3)
    axes[2].set_ylim(0, 1)

    for bar, value in zip(bars3, results_df['Correlação']):
        axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    f'{value:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

def perform_statistical_analysis(results):
    """Realiza análise estatística detalhada dos resultados"""

    print("\n5.2 ANÁLISE ESTATÍSTICA DETALHADA")
    print("-" * 40)

    for name, result in results.items():
        residuals = result['y_true'] - result['y_pred']

        print(f"\n{name}:")
        print(f"  Média dos resíduos: {np.mean(residuals):.4f}")
        print(f"  Desvio padrão dos resíduos: {np.std(residuals):.4f}")
        print(f"  Assimetria dos resíduos: {skew(residuals):.4f}")
        print(f"  Curtose dos resíduos: {kurtosis(residuals):.4f}")
        print(f"  R² ajustado: {result['Correlação']**2:.4f}")

# =============================================================================
# 6. ANÁLISE DE IMPORTÂNCIA DAS VARIÁVEIS
# =============================================================================

def analyze_feature_importance(X, y):
    """Analisa importância das variáveis usando Random Forest"""

    print("\n" + "="*60)
    print("6. ANÁLISE DE IMPORTÂNCIA DAS VARIÁVEIS")
    print("="*60)

    # Treinamento do Random Forest para análise de importância
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X, y)

    # Importância das variáveis
    feature_importance = pd.DataFrame({
        'Variável': X.columns,
        'Importância': rf_model.feature_importances_
    }).sort_values('Importância', ascending=False).reset_index(drop=True)

    print("Importância das Variáveis (Random Forest):")
    print(feature_importance.round(4).to_string(index=False))

    # Visualização
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance, y='Variável', x='Importância', palette='viridis')
    plt.title('Importância das Variáveis - Random Forest')
    plt.xlabel('Importância Relativa')
    plt.grid(True, alpha=0.3)

    # Adicionar valores nas barras
    for i, (var, imp) in enumerate(zip(feature_importance['Variável'], feature_importance['Importância'])):
        plt.text(imp + 0.005, i, f'{imp:.3f}', va='center')

    plt.tight_layout()
    plt.show()

    return feature_importance

# =============================================================================
# 7. ANÁLISE DE RESÍDUOS
# =============================================================================

def analyze_residuals(results, best_model):
    """Análise detalhada dos resíduos do melhor modelo"""

    print("\n" + "="*60)
    print("7. ANÁLISE DE RESÍDUOS")
    print("="*60)

    best_results = results[best_model]
    residuals = best_results['y_true'] - best_results['y_pred']
    y_pred = best_results['y_pred']

    print(f"Análise de resíduos para: {best_model}")

    # Figura: Análise de resíduos
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Resíduos vs Preditos
    axes[0, 0].scatter(y_pred, residuals, alpha=0.6, color='steelblue')
    axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.8)
    axes[0, 0].set_xlabel('Valores Preditos')
    axes[0, 0].set_ylabel('Resíduos')
    axes[0, 0].set_title('Resíduos vs Valores Preditos')
    axes[0, 0].grid(True, alpha=0.3)

    # Histograma dos resíduos
    axes[0, 1].hist(residuals, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[0, 1].set_xlabel('Resíduos')
    axes[0, 1].set_ylabel('Frequência')
    axes[0, 1].set_title('Distribuição dos Resíduos')
    axes[0, 1].axvline(np.mean(residuals), color='red', linestyle='--',
                      label=f'Média: {np.mean(residuals):.3f}')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Q-Q Plot dos resíduos
    stats.probplot(residuals, dist="norm", plot=axes[0, 2])
    axes[0, 2].set_title('Q-Q Plot - Normalidade dos Resíduos')
    axes[0, 2].grid(True, alpha=0.3)

    # Resíduos padronizados
    std_residuals = residuals / np.std(residuals)
    axes[1, 0].scatter(y_pred, std_residuals, alpha=0.6, color='orange')
    axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.8)
    axes[1, 0].axhline(y=2, color='red', linestyle=':', alpha=0.6, label='±2σ')
    axes[1, 0].axhline(y=-2, color='red', linestyle=':', alpha=0.6)
    axes[1, 0].set_xlabel('Valores Preditos')
    axes[1, 0].set_ylabel('Resíduos Padronizados')
    axes[1, 0].set_title('Resíduos Padronizados')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Resíduos absolutos vs Preditos
    abs_residuals = np.abs(residuals)
    axes[1, 1].scatter(y_pred, abs_residuals, alpha=0.6, color='purple')
    axes[1, 1].set_xlabel('Valores Preditos')
    axes[1, 1].set_ylabel('|Resíduos|')
    axes[1, 1].set_title('Resíduos Absolutos vs Preditos')
    axes[1, 1].grid(True, alpha=0.3)

    # Box plot dos resíduos
    axes[1, 2].boxplot(residuals)
    axes[1, 2].set_title('Box Plot dos Resíduos')
    axes[1, 2].set_ylabel('Resíduos')
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Teste de normalidade dos resíduos
    from scipy.stats import shapiro, jarque_bera

    print(f"\nTestes de Normalidade dos Resíduos:")

    # Teste de Shapiro-Wilk
    shapiro_stat, shapiro_p = shapiro(residuals)
    print(f"Shapiro-Wilk: estatística = {shapiro_stat:.4f}, p-valor = {shapiro_p:.4f}")

    # Teste de Jarque-Bera
    jb_stat, jb_p = jarque_bera(residuals)
    print(f"Jarque-Bera: estatística = {jb_stat:.4f}, p-valor = {jb_p:.4f}")

    # Identificar outliers
    outliers = np.where(np.abs(std_residuals) > 2)[0]
    print(f"\nOutliers identificados (|resíduo padronizado| > 2): {len(outliers)} observações")
    if len(outliers) > 0:
        print(f"Índices dos outliers: {outliers[:10]}{'...' if len(outliers) > 10 else ''}")

# =============================================================================
# 8. FUNÇÃO PRINCIPAL
# =============================================================================

def main():
    """Função principal que executa toda a análise"""

    print("ANÁLISE COMPLETA DO BOSTON HOUSING DATASET")
    print("=" * 80)
    print("Disciplina: ELT 573 – Introdução ao Aprendizado Estatístico")
    print("Comparação de Modelos com Validação Cruzada Leave-One-Out")
    print("=" * 80)

    try:
        # 1. Carregamento dos dados
        X, y, boston_data = load_and_prepare_data()

        # 2. Análise exploratória
        correlation_matrix, correlations_target = exploratory_data_analysis(X, y)

        # 3. Configuração dos modelos
        models = setup_models()

        # 4. Validação cruzada LOOCV
        results = perform_loocv_analysis(X, y, models)

        # 5. Análise dos resultados
        results_df, best_model = analyze_results(results)

        # 6. Importância das variáveis
        feature_importance = analyze_feature_importance(X, y)

        # 7. Análise de resíduos
        analyze_residuals(results, best_model)

        # 8. Relatório final
        print("\n" + "="*80)
        print("RELATÓRIO FINAL")
        print("="*80)
        print(f"Dataset: {X.shape[0]} observações, {X.shape[1]} variáveis")
        print(f"Melhor modelo: {best_model}")
        print(f"MSE do melhor modelo: {results[best_model]['MSE']:.3f}")
        print(f"Correlação do melhor modelo: {results[best_model]['Correlação']:.3f}")
        print(f"Variável mais importante: {feature_importance.iloc[0]['Variável']}")
        print("="*80)
        print("Análise concluída com sucesso!")

        return results, results_df, feature_importance

    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None

# =============================================================================
# EXECUÇÃO
# =============================================================================

if __name__ == "__main__":
    results, results_df, feature_importance = main()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: